In [ ]:
import nltk
import numpy as np
import pickle as pkl
import pandas as pd
import copy
import time
import os
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
class_mapping = {}
with open('dbpedia_csv/classes.txt') as txt_file:
    line_num = 1
    for line in txt_file:
        class_mapping[line_num] = line.strip()
        line_num+=1
        
print(class_mapping)

In [ ]:
train_data = pd.read_csv('dbpedia_csv/train.csv', header = -1, names = ['ClassIndex', 'Title', 'Content'])
test_data = pd.read_csv('dbpedia_csv/test.csv', header = -1, names = ['ClassIndex', 'Title', 'Content'])

In [ ]:
training_array = train_data.drop(labels = 'Title', axis = 1).values
test_array = test_data.drop(labels = 'Title', axis = 1).values

### Create training dataset

In [ ]:
ID_count = 0
partition = {}
partition['train'] = []
partition['validation'] = []
labels = {}

start = time.time()
for i in range(len(class_mapping)):
    class_index = i+1
    train_count = 0
    num_sent = 0
    sample_vec = []
    for j in range(len(training_array)):
        if training_array[j,0] == class_index:
            par_vec = []
            for cur_word in word_tokenize(training_array[j,1]):
                par_vec.append(cur_word)
                sample_vec.append(copy.copy(par_vec))
                if (train_count+1)%32 == 0:
                    pkl.dump((class_index, sample_vec), open('Training/'+str(ID_count), 'wb'))
                    labels[str(ID_count)] = class_index
                    sample_vec = []
                    if num_sent < 35000:
                        partition['train'].append(str(ID_count))
                    else:
                        partition['validation'].append(str(ID_count))
                    ID_count += 1
                train_count += 1
            num_sent += 1
    print('Elapsed time: {}'.format((time.time()-start)/60))
pkl.dump((partition, labels),open('Training/Metadata', 'wb'))

### Create testing dataset

In [ ]:
start = time.time()
for i in range(len(class_mapping)):
    class_index = i+1
    test_count = 0
    for j in range(len(test_array)):
        if test_array[j,0] == class_index:
            par_vec = []
            for cur_word in word_tokenize(test_array[j,1]):
                par_vec.append(cur_word)
                sample_len = len(par_vec)
                if not os.path.exists('Testing/Class{}/{}'.format(class_index,sample_len)):
                    os.makedirs('Testing/Class{}/{}'.format(class_index,sample_len))
                pkl.dump((class_index, par_vec), open('Testing/Class{}/{}/{}'.format(class_index, sample_len, test_count),'wb'))
                test_count += 1
    print('Elapsed time: {}'.format((time.time()-start)/60))